## Seach optimal augmentation intensity by $\mu$ (minimal mutual information)

In [1]:
import os
os.chdir("..")
print(os.getcwd())
assert os.getcwd() == "/home1/quanquan/code/landmark/code/tproj"

/home1/quanquan/code/landmark/code/tproj


In [6]:
"""
    search_optimal_aug_params
"""
import cv2
from sklearn.metrics import mutual_info_score
from scipy.stats import entropy
# from scipy.misc import imread
import numpy as np
import cv2
from skimage.exposure import histogram
from scipy.stats import entropy
import matplotlib
from tutils import tfilename
from PIL import Image
import torchvision.transforms.functional as F
from scipy.stats import norm
matplotlib.use('Agg')  # Must be before importing matplotlib.pyplot or pylab!
import matplotlib.pyplot as plt
plt.style.use('ggplot')


EX_CONFIG = {
    'dataset': {
        'name': 'Cephalometric',
        'pth': '/home1/quanquan/datasets/Cephalometric/',
        'entr': '/home1/quanquan/datasets/Cephalometric/entr1/train/',
        'prob': '/home1/quanquan/datasets/Cephalometric/prob/train/',
        'n_cls': 19,
    }
}


def search_mi_for_all_patch():
    from datasets.ceph.ceph_ssl import Test_Cephalometric
    pth = '/home1/quanquan/datasets/Cephalometric/'
    testset = Test_Cephalometric(pth, mode="Train")
    # item, landmark_list, template_patches = testset.__getitem__(0)
    # data = testset.__getitem__(id_oneshot)
    landmark_list = []
    for i in range(len(testset)):
        landmark = testset.ref_landmarks(i)
        landmark_list.append(landmark)

    pth = EX_CONFIG['dataset']['pth']
    m0_list = []
    m1_list = []
    r_list = []
    en_list = []
    for i in range(150):
        landmarks = landmark_list[i]
        tmp_list0 = []
        tmp_list1 = []
        tmp_list2 = []
        tmp_list3 = []
        for j, landmark in enumerate(landmarks):
            print("Processing ", i, j, end="\r")
            im_pth = tfilename(pth, "RawImage/TrainingData", f"{i+1:03d}.bmp")
            # print(im_pth)
            im = cv2.imread(im_pth, cv2.IMREAD_GRAYSCALE)
            im = cv2.resize(im, (384, 384))
            lm = landmark
            ps_half = 32
            patch = im[max(lm[0]-ps_half, 0):lm[0]+ps_half, max(lm[1]-ps_half, 0):lm[1]+ps_half]
            mi0, mi, ratio, e1, e2 = _test_mi_fn(patch)
            tmp_list0.append(mi0)
            tmp_list1.append(mi)
            tmp_list2.append(ratio)
            tmp_list3.append(e1)
        m0_list.append(tmp_list0)
        m1_list.append(tmp_list1)
        r_list.append(tmp_list2)
        en_list.append(tmp_list3)
    m0 = np.array(m0_list)
    m1 = np.array(m1_list)
    rr = np.array(r_list)
    en = np.array(en_list)
    print(m0.mean(), m0.std(), m0.mean(axis=0), m0.std(axis=0))  # 2.781 +- 0.686
    print(m1.mean(), m1.std(), m1.mean(axis=0), m1.std(axis=0))  # 0.833 +- 0.236
    print(rr.mean(), rr.std(), rr.mean(axis=0), rr.std(axis=0))  # 0.302 +- 0.07
    print(en.mean(), en.std(), en.mean(axis=0), en.std(axis=0))  # 4.593 +- 0.506
    np.save('./cache/mi_ceph_all.npy', [m0, m1, rr, en])


def _test_mi_fn(patch, br=1.6, ct=1.5):
    assert len(patch.shape) == 2, f"Got {patch.shape}"
    assert patch.shape[0] >0 and patch.shape[1] > 0, f"Got {patch.shape}"
    patch_aug = Image.fromarray(patch)
    patch_aug = F.adjust_brightness(patch_aug, br)
    patch_aug = F.adjust_contrast(patch_aug, ct)
    patch_aug = np.array(patch_aug)
    assert patch_aug.shape[0] >0 and patch_aug.shape[1] > 0, f"Got {patch_aug.shape}"
    fea1 = get_fea(patch)
    fea2 = get_fea(patch_aug)
    mi0 = mutual_info_score(fea1.copy(), fea1.copy())
    mi = mutual_info_score(fea1, fea2)
    entr1 = entropy(fea1)
    entr2 = entropy(fea2)
    return mi0, mi, mi/mi0, entr1, entr2

def get_fea(patch):
    fea = np.zeros((256,))
    hist, idx = histogram(patch, nbins=256)
    for hi, idi in zip(hist, idx):
        # print(hi, idi, i, j)
        fea[idi] = hi
    return fea

In [7]:
search_aug_for_all_patch()

The sizes are set as  [384, 384]
Initializing Datasets: (split:'Train') (len:(150)) 
2.781115538153201 0.6866188393380371 [2.51138927 3.18064627 3.1936682  2.11334156 3.26182041 2.72902131
 2.16312098 1.94328915 2.01532239 3.26112403 3.20813622 3.16552408
 3.12697808 2.64613941 3.25124197 1.97482131 3.206975   3.22484813
 2.66378744] [0.46076691 0.30574453 0.2845213  0.48100281 0.24713011 0.75816679
 0.75060892 0.65067037 0.6793318  0.23704422 0.36165119 0.38236706
 0.30267382 0.60297583 0.24602729 0.60459519 0.21163927 0.24409171
 0.47145102]
0.8334361740522674 0.236187758371206 [0.58855864 0.9689842  0.88423108 0.41595766 0.93467499 0.87520715
 0.75355031 0.69097756 0.70706193 0.9617875  0.95781572 0.96094506
 0.98639255 0.89073955 0.96499311 0.71407015 0.93329412 0.91891464
 0.7271314 ] [0.26388173 0.12006105 0.12696776 0.31245908 0.10358765 0.20422783
 0.21599622 0.18820669 0.19157507 0.13725611 0.11465392 0.12241823
 0.11602296 0.19213057 0.09218982 0.18752875 0.1210562  0.1017045